In [9]:
import pandas as pd
from pathlib import Path
from scipy import odr
import numpy as np

In [2]:
DATA_PATH = Path("./laser_labels_cleaned.csv")

DATA_PATH.exists()

True

In [3]:
df = pd.read_csv(DATA_PATH)

df

,dive_id,image_id,image_path,label_x,label_y,label
0,16,1866,2025-02-10 REEF Data Dump SMILE 6/E4E Research...,1944.462958,1374.748830,Red Laser
1,16,1867,2025-02-10 REEF Data Dump SMILE 6/E4E Research...,1918.782168,1290.050424,Red Laser
2,16,1868,2025-02-10 REEF Data Dump SMILE 6/E4E Research...,1931.771859,1333.988340,Red Laser
3,35,1872,2023.08.03.FishSense.FSL-01D/GraySnapper01_Mol...,1941.084918,1333.166681,Red Laser
4,35,1873,2023.08.03.FishSense.FSL-01D/GraySnapper01_Mol...,1943.358143,1340.802341,Red Laser
...,...,...,...,...,...,...
37806,471,129445,2025-03-13 Reef Data Dump/121024_Alligator_FSL...,2071.000000,1585.000000,Green Laser
37807,471,129446,2025-03-13 Reef Data Dump/121024_Alligator_FSL...,2100.000000,1663.000000,Green Laser
37808,471,129447,2025-03-13 Reef Data Dump/121024_Alligator_FSL...,2076.000000,1576.000000,Green Laser
37809,471,129448,2025-03-13 Reef Data Dump/121024_Alligator_FSL...,2087.000000,1630.000000,Green Laser


In [6]:
dive_ids = df['dive_id'].unique()

len(dive_ids)

262

In [8]:
def f(B, x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0]*x + B[1]

In [16]:
all_s_normalized = []

for dive_id in dive_ids:
    dive_data = df[df['dive_id'] == dive_id]

    s_noisy = np.zeros((3, len(dive_data)))
    s_noisy[0, :] = dive_data['label_x'].to_numpy()
    s_noisy[1, :] = dive_data['label_y'].to_numpy()
    s_noisy[2, :] = 1  # Homogeneous coordinates

    model = odr.Model(f)
    data = odr.Data(s_noisy[0, :], s_noisy[1, :])
    odr_run = odr.ODR(data, model, beta0=[1.0, 1.0])
    results = odr_run.run()

    s_reconstructed = np.zeros_like(s_noisy)
    s_reconstructed[0, :] = results.xplus
    s_reconstructed[1, :] = results.y
    s_reconstructed[2, :] = 1

    s_normalized = s_reconstructed - s_noisy
    all_s_normalized.append(s_normalized)

all_s_normalized = np.hstack(all_s_normalized)

calculated_mean = np.mean(all_s_normalized, axis=1)
calculated_std = np.std(all_s_normalized, axis=1)

calculated_mean, calculated_std

(array([-0.00460133,  0.03549944,  0.        ]),
 array([2.98374702, 5.07608042, 0.        ]))